In [2]:
from mxnet import nd

/home/wk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/wk/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [3]:
# 输入输出数据格式是 batch x channel x height x width，这里batch和channel都是1
# 权重格式是 output_channels x in_channels x height x width，这里input_filter和output_filter都是1。
##conv layers
w = nd.arange(4).reshape((1,1,2,2))
b = nd.array([1])
#input 1*1*3*3
data = nd.arange(9).reshape((1,1,3,3))
print(w.shape[2:])
out = nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[1])
print('input:',data,'\nweight:',w,'\nbias:',b,'\noutput:',out)

(2, 2)
input: 
[[[[0. 1. 2.]
   [3. 4. 5.]
   [6. 7. 8.]]]]
<NDArray 1x1x3x3 @cpu(0)> 
weight: 
[[[[0. 1.]
   [2. 3.]]]]
<NDArray 1x1x2x2 @cpu(0)> 
bias: 
[1.]
<NDArray 1 @cpu(0)> 
output: 
[[[[20. 26.]
   [38. 44.]]]]
<NDArray 1x1x2x2 @cpu(0)>


In [4]:
out = nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[1],stride=(2,2),pad=(1,1))
print('input:',data,'\nweight:',w,'\nbias:',b,'\noutput:',out)

input: 
[[[[0. 1. 2.]
   [3. 4. 5.]
   [6. 7. 8.]]]]
<NDArray 1x1x3x3 @cpu(0)> 
weight: 
[[[[0. 1.]
   [2. 3.]]]]
<NDArray 1x1x2x2 @cpu(0)> 
bias: 
[1.]
<NDArray 1 @cpu(0)> 
output: 
[[[[ 1.  9.]
   [22. 44.]]]]
<NDArray 1x1x2x2 @cpu(0)>


In [5]:
w = nd.arange(16).reshape((2,2,2,2))
data = nd.arange(18).reshape((1,2,3,3))
b = nd.array([1,2])

out = nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[0])
print('input:',data,'\nweight:',w,'\nbias:',b,'\noutput:',out)

input: 
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)> 
weight: 
[[[[ 0.  1.]
   [ 2.  3.]]

  [[ 4.  5.]
   [ 6.  7.]]]


 [[[ 8.  9.]
   [10. 11.]]

  [[12. 13.]
   [14. 15.]]]]
<NDArray 2x2x2x2 @cpu(0)> 
bias: 
[1. 2.]
<NDArray 2 @cpu(0)> 
output: 
[[[[ 269.  297.]
   [ 353.  381.]]

  [[ 686.  778.]
   [ 962. 1054.]]]]
<NDArray 1x2x2x2 @cpu(0)>


## pooling layer

In [6]:
data = nd.arange(18).reshape((1,2,3,3))
max_pool = nd.Pooling(data,pool_type='max',kernel=(2,2))
avg_pool = nd.Pooling(data, pool_type='avg',kernel=(2,2))
print('input:',data,'\nmax pool:',max_pool,'\navg pool:',avg_pool)

input: 
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]

  [[ 9. 10. 11.]
   [12. 13. 14.]
   [15. 16. 17.]]]]
<NDArray 1x2x3x3 @cpu(0)> 
max pool: 
[[[[ 4.  5.]
   [ 7.  8.]]

  [[13. 14.]
   [16. 17.]]]]
<NDArray 1x2x2x2 @cpu(0)> 
avg pool: 
[[[[ 2.  3.]
   [ 5.  6.]]

  [[11. 12.]
   [14. 15.]]]]
<NDArray 1x2x2x2 @cpu(0)>


In [7]:
import sys
# sys.path.append('/home/wk/gluon_ws')
from mxnet import gluon
from utils1 import load_data_fashion_mnist


In [8]:
batch_size = 256
train_data, test_data = load_data_fashion_mnist(batch_size)


/home/wk/anaconda3/lib/python3.6/site-packages/mxnet/gluon/data/vision.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/home/wk/anaconda3/lib/python3.6/site-packages/mxnet/gluon/data/vision.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


In [9]:
import mxnet as mx
try: 
    ctx = mx.gpu()
    _ = nd.zeros((1,),ctx=ctx)
except:
    ctx = mx.cpu()
ctx

gpu(0)

In [10]:
weight_scale = .01
#output channels = 20, kernel = (5,5)
w1 = nd.random_normal(shape=(20,1,5,5),scale=weight_scale,ctx=ctx)
b1 = nd.zeros(w1.shape[0],ctx=ctx)

#output channels = 50, kernel = (3,3)
w2 = nd.random_normal(shape=(50,20,3,3),scale=weight_scale,ctx=ctx)
b2 = nd.zeros(w2.shape[0],ctx=ctx)

#output dim = 128
w3 = nd.random_normal(shape=(1250,128),scale = weight_scale,ctx=ctx)
b3 = nd.zeros(w3.shape[1],ctx=ctx)

#output dim = 10
w4 = nd.random_normal(shape=(w3.shape[1],10),scale=weight_scale,ctx=ctx)
b4 = nd.zeros(w4.shape[1],ctx=ctx)

params = [w1,b1,w2,b2,w3,b3,w4,b4]
for param in params:
    param.attach_grad()

In [11]:
def net(X,verbose=False):
    print(X.shape)
    X = X.as_in_context(w1.context)
    #conv1
    h1_conv = nd.Convolution(data = X,weight=w1, bias=b1,kernel=w1.shape[2:],num_filter=w1.shape[0])
    h1_activation = nd.relu(h1_conv)
    h1 = nd.Pooling(data=h1_activation,pool_type='max',kernel=(2,2),stride=(2,2))
    
    #conv2
    h2_conv=nd.Convolution(data=h1,weight=w2,bias=b2,kernel=w2.shape[2:],num_filter=w2.shape[0])
    h2_activation = nd.relu(h2_conv)
    h2  = nd.Pooling(data=h2_activation,pool_type='max',kernel=(2,2),stride=(2,2))
    h2 = nd.flatten(h2)
    #fc1
    h3_linear = nd.dot(h2,w3)+b3
    h3 = nd.relu(h3_linear)
    #fc2
    h4_linear = nd.dot(h3,w4)+b4
    if verbose:
        print('1st conv block:', h1.shape)
        print('2nd conv block:', h2.shape)
        print('1st dense:', h3.shape)
        print('2nd dense:', h4_linear.shape)
        print('output:', h4_linear)
    return h4_linear

In [12]:
for data, _ in train_data:
    net(data, verbose=True)
    break

(256, 1, 28, 28)
1st conv block: (256, 20, 12, 12)
2nd conv block: (256, 1250)
1st dense: (256, 128)
2nd dense: (256, 10)
output: 
[[-1.34742062e-04  3.91600188e-05  4.63903853e-05 ...  2.09621561e-04
   9.39194942e-05  5.92381475e-05]
 [-2.03683696e-04  6.27324116e-05  1.72832137e-04 ...  1.50834210e-04
   1.32468296e-04  1.49707019e-04]
 [-1.30133485e-04  2.19544054e-05  8.49038915e-05 ...  6.50751317e-05
   1.35110182e-04  1.06310552e-04]
 ...
 [-1.35622919e-04 -2.35706611e-05  6.91858877e-05 ...  1.73665103e-04
   8.14187661e-05  1.10244604e-04]
 [-7.69861799e-05  3.55988450e-05  5.04717973e-05 ...  3.76938515e-05
   7.65914592e-05  5.31316109e-05]
 [-1.24010185e-04  4.00239878e-05  1.66009529e-04 ...  8.12018043e-05
   1.13639304e-04  1.36643881e-04]]
<NDArray 256x10 @gpu(0)>


In [86]:
import mxnet as mx
import time
s = nd.arange(10000).reshape((10000,1))
# print(s)
a = nd.random_normal(shape=(10000,10000),scale=.01)
# print(a)
now1 = time.time()
f = nd.dot(a,s)
now2 = time.time()
print((now2-now1)*1000)
now11 = time.time()
aa = a.as_in_context(mx.gpu())
now22 = time.time()
print((now22-now11)*1000)


0.07486343383789062
0.1895427703857422


In [73]:
s = nd.arange(100,ctx=mx.gpu()).reshape((100,1))
# print(s)
a = nd.random_normal(shape=(100,100),scale=.01,ctx=mx.gpu())
# print(a)
now1 = time.time()
f = nd.dot(a,s)
now2 = time.time()
print((now2-now1)*1000)

0.08845329284667969


In [102]:
from mxnet.gluon import nn

In [125]:
def get_net():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(10,activation='relu'))
        net.add(nn.Dense(2))
    return net

net = get_net()
# print(net.collect_params)
net.initialize()
x = nd.random.uniform(shape=(2,10))
net(x)
print(x)



[[0.10232294 0.9970644  0.07205658 0.43203783 0.8144738  0.16743097
  0.05625672 0.509534   0.5285208  0.38097662]
 [0.322278   0.42643943 0.77552855 0.9932089  0.19089413 0.76034296
  0.5067027  0.78196734 0.47605646 0.9856647 ]]
<NDArray 2x10 @cpu(0)>


In [142]:
print(net(x))
# print(net.collect_params)
params = net.collect_params()
print(params.keys())
print(params.get('dense0_weight').data())
print(params.get('dense1_weight').data())


[[-0.00171718  0.00589358]
 [ 0.00011509 -0.00059731]]
<NDArray 2x2 @cpu(0)>
odict_keys(['sequential18_dense0_weight', 'sequential18_dense0_bias', 'sequential18_dense1_weight', 'sequential18_dense1_bias'])

[[ 0.01517928 -0.02047256 -0.02355406  0.04838201 -0.0278985  -0.01531811
   0.02635069 -0.02959068 -0.03498466 -0.05543771]
 [-0.03848986 -0.04299378 -0.05290868  0.02053718  0.00883086  0.03420109
   0.00150619  0.01088265 -0.00421003 -0.06297817]
 [-0.00487888  0.05704216  0.03639957  0.04210176  0.04339372 -0.01837374
   0.03754044 -0.04624455  0.01866682 -0.05854589]
 [-0.00147696  0.01595342  0.02077559 -0.03274477 -0.03626035 -0.04805061
   0.05141753 -0.03726027 -0.00394321 -0.00953716]
 [ 0.06256447 -0.01444218 -0.00663622  0.06836209  0.05412047  0.02242351
   0.05974532  0.00864073 -0.05862761 -0.03281445]
 [ 0.04172114  0.03622857  0.01542428  0.00313994 -0.04792007  0.03810354
   0.02416857 -0.06902966 -0.03550147  0.0334644 ]
 [ 0.04488035 -0.05207274 -0.05375268 -0.0

In [149]:
filename1 = './mlp.params'
net.save_params(filename1)
net2 = get_net()
net2.load_params(filename1,mx.cpu())
print(net2(x))


[[-0.00171718  0.00589358]
 [ 0.00011509 -0.00059731]]
<NDArray 2x2 @cpu(0)>


In [156]:
from mxnet import nd
from mxnet.gluon import nn
class CenterLayer(nn.Block):
    def __init__(self,**k):
        super(CenterLayer, self).__init__(**k)
    def forward(self, x):
        return x - x.mean()

In [157]:
layer  = CenterLayer()
layer(nd.array([1,2,3,4,5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [158]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(128))
    net.add(nn.Dense(10))
    net.add(CenterLayer())

In [182]:
net.initialize()
y = net(nd.random.uniform(shape=(4,8)))
# print(y)
y.mean()
param = net.collect_params()
print(param.keys())
print(param.get('dense0_weight').data())
print(param.get('dense0_bias').data())
print(param.get('dense1_weight').data())

odict_keys(['sequential23_dense0_weight', 'sequential23_dense0_bias', 'sequential23_dense1_weight', 'sequential23_dense1_bias'])

[[-0.00811242 -0.01717252  0.04150712 ... -0.01934163 -0.02216569
   0.04895021]
 [ 0.05007882 -0.04186276  0.01005154 ... -0.00939962 -0.06334607
  -0.01427951]
 [-0.00195836  0.06544624  0.05563306 ... -0.05469994  0.00990371
   0.03593857]
 ...
 [-0.03689339 -0.03231623  0.02224629 ...  0.03027458  0.03328106
   0.02344458]
 [ 0.04445826 -0.04597376  0.04472021 ...  0.00134477  0.00602937
   0.0187114 ]
 [ 0.03419255  0.00320629  0.04187601 ... -0.01281732 -0.03971042
  -0.00735135]]
<NDArray 128x8 @cpu(0)>

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

/home/wk/anaconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential23_dense0_weight is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wk/anaconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential23_dense0_bias is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wk/anaconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential23_dense1_weight is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/home/wk/anaconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter sequential23_dense1_bias is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-init

In [198]:
my_param = gluon.Parameter("wk_test",shape = (3,3))
my_param.initialize()
(my_param.data(),my_param.grad())

(
 [[-0.04125438 -0.06554382 -0.01985936]
  [-0.04728559 -0.00772712  0.05879719]
  [-0.01851068  0.0688391  -0.04979665]]
 <NDArray 3x3 @cpu(0)>, 
 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]
 <NDArray 3x3 @cpu(0)>)

In [202]:
pd = gluon.ParameterDict(prefix="block1_")
pd.get("wk_test",shape=(3,3))
pd

KeyError: 'shape'